In [1]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import pandas as pd

In [3]:
pd.set_option("display.max_columns", 500)

In [4]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [5]:
rk = pd.read_csv("rk_fifa_2022_10_06.csv", sep="\t")

<IPython.core.display.Javascript object>

In [6]:
rk.head()

,RK,Team,Total Points,Previous Points
0,1,Brazil,1841.30,1837.56
1,2,Belgium,1816.71,1821.92
2,3,Argentina,1773.88,1770.65
3,4,France,1759.78,1764.85
4,5,England,1728.47,1737.46


<IPython.core.display.Javascript object>

In [7]:
rk.describe()

,RK,Total Points,Previous Points
count,211.00000,211.000000,211.000000
mean,106.00000,1220.687346,1220.645592
std,61.05462,253.420954,253.331712
min,1.00000,762.220000,763.820000
25%,53.50000,1007.675000,1008.330000
50%,106.00000,1192.090000,1183.900000
75%,158.50000,1433.465000,1431.790000
max,211.00000,1841.300000,1837.560000


<IPython.core.display.Javascript object>

In [8]:
def get_rk(team=""):
    try:
        return int(rk.loc[rk["Team"] == team]["RK"])
    except TypeError:
        print("get_rk(): Team not found")

<IPython.core.display.Javascript object>

In [9]:
def get_points(team="", previous=False):
    try:
        if previous:
            return float(rk.loc[rk["Team"] == team]["Total Points"])
        else:
            return float(rk.loc[rk["Team"] == team]["Previous Points"])
    except TypeError:
        print("get_points(): Team not found")

<IPython.core.display.Javascript object>

In [10]:
def get_probability_of_win(team=""):
    ## Probability of defeat any team
    # There are ([number of teams] - [RK]) teams that are likely to defeat
    p_any = 1 - get_rk(team) / len(rk)

    ## Points increase
    increase = get_points(team) / get_points(team, previous=True)

    ## Combined
    p = p_any * increase

    return p

<IPython.core.display.Javascript object>

In [11]:
def how_many_goals(team=""):
    goals = 0

    if get_rk(team) <= 10:
        goals = 3
    elif get_rk(team) > 10 and get_rk(team) <= 20:
        goals = 2
    elif get_rk(team) > 20 and get_rk(team) <= 30:
        goals = 1
    else:
        goals = 0

    return goals

<IPython.core.display.Javascript object>

In [12]:
def estimate_result(team_1="", team_2=""):
    # Probability of win
    p_team_1 = get_probability_of_win(team_1)
    p_team_2 = get_probability_of_win(team_2)

    print(
        f"Team 1: {team_1} [RK: {get_rk(team_1)}, Points: {get_points(team_1)}], probability of win: {p_team_1}"
    )
    print(
        f"Team 2: {team_2} [RK: {get_rk(team_2)}, Points: {get_points(team_2)}], probability of win: {p_team_2}"
    )

    # Difference
    diff = abs(p_team_1 - p_team_2)
    print(f"Diff: {diff}")

    # How many goals?
    team_1_goals = how_many_goals(team_1)
    team_2_goals = how_many_goals(team_2)

    plus_minus_winner_goals = 0
    plus_minus_loser_goals = 0

    if min(p_team_1, p_team_2) > 0.87:
        if diff <= 0.05:
            plus_minus_winner_goals = -2
            plus_minus_loser_goals = -2
        elif diff > 0.05 and diff <= 0.1:
            plus_minus_winner_goals = -1
            plus_minus_loser_goals = -1
        elif diff > 0.1 and diff <= 0.15:
            plus_minus_winner_goals = 0
            plus_minus_loser_goals = -2
        elif diff > 0.15:
            plus_minus_winner_goals = 1
            plus_minus_loser_goals = -3
    else:
        if diff <= 0.05:
            plus_minus_winner_goals = 0
            plus_minus_loser_goals = 0
        elif diff > 0.05 and diff <= 0.1:
            plus_minus_winner_goals = 1
            plus_minus_loser_goals = 0
        elif diff > 0.1 and diff <= 0.15:
            plus_minus_winner_goals = 1
            plus_minus_loser_goals = -1
        elif diff > 0.15:
            plus_minus_winner_goals = 2
            plus_minus_loser_goals = -1

    # Winner / Loser
    if p_team_1 > p_team_2:
        # winner team_1
        team_1_goals = team_1_goals + plus_minus_winner_goals
        team_2_goals = team_2_goals + plus_minus_loser_goals

    elif p_team_1 < p_team_2:
        # winner team_2
        team_2_goals = team_2_goals + plus_minus_winner_goals
        team_1_goals = team_1_goals + plus_minus_loser_goals

    else:
        team_1_goals = team_1_goals + 0
        team_2_goals = team_2_goals + 0

    if team_1_goals < 0:
        team_1_goals = 0

    if team_2_goals < 0:
        team_2_goals = 0

    print(f"Match result: {team_1} {team_1_goals}-{team_2_goals} {team_2}")

<IPython.core.display.Javascript object>

In [13]:
estimate_result("Poland", "Mexico")

Team 1: Poland [RK: 26, Points: 1546.18], probability of win: 0.8754127627305917
Team 2: Mexico [RK: 13, Points: 1649.57], probability of win: 0.9410585055040054
Diff: 0.06564574277341373
Match result: Poland 0-1 Mexico


<IPython.core.display.Javascript object>

In [14]:
estimate_result("Poland", "Saudi Arabia")

Team 1: Poland [RK: 26, Points: 1546.18], probability of win: 0.8754127627305917
Team 2: Saudi Arabia [RK: 51, Points: 1435.74], probability of win: 0.7572179305655462
Diff: 0.11819483216504545
Match result: Poland 2-0 Saudi Arabia


<IPython.core.display.Javascript object>

In [15]:
estimate_result("Poland", "Argentina")

Team 1: Poland [RK: 26, Points: 1546.18], probability of win: 0.8754127627305917
Team 2: Argentina [RK: 3, Points: 1770.65], probability of win: 0.9839870123777187
Diff: 0.10857424964712703
Match result: Poland 0-3 Argentina


<IPython.core.display.Javascript object>